In [1]:
import pandas as pd

# Function to resample the 1 minute timeframe data to new timeframe
def resample_data(data, timeframe):
    
    resampled_data = data.resample(timeframe).agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    }).dropna()
    
    return resampled_data

# Function to find high and low prices for specific time ranges within a day
def find_high_low(df, time_ranges):
    results = []
    daily_data = df.resample('D')

    for date, group in daily_data:
        result = {'Date': date.date(), 'Day of the Week': date.strftime('%A')}

        for start_time, end_time in time_ranges:
            intraday_group = group.between_time(start_time, end_time)
            if not intraday_group.empty:
                result[f'{start_time}-{end_time} High'] = intraday_group['High'].max()
                result[f'{start_time}-{end_time} Low'] = intraday_group['Low'].min()

        results.append(result)

    # Create DataFrame and drop rows with 'Friday' as Day of the Week
    results_df = pd.DataFrame(results)
    results_df = results_df[results_df['Day of the Week'] != 'Friday']
    results_df = results_df[results_df['Day of the Week'] != 'Saturday']
    return results_df

# Function to find the first candle closing above or below the high and low after the 19:30-20:30 time range
def find_first_candle(df, time_ranges):
    high_low_data = find_high_low(df, time_ranges)
    
    results = []
    
    # Get the first time range's end time
    _, first_end_time = time_ranges[0]
    
    for _, row in high_low_data.iterrows():
        date = row['Date']
        day_group = df.loc[df.index.date == date]
        
        # Filter data to only include times after the end of the first time range
        day_group = day_group.between_time(first_end_time, '23:59:59')
        
        # Check for the first candle that crosses the 19:30-20:30 High or Low
        for idx, candle in day_group.iterrows():
            condition = None
            if candle['Close'] > row['19:30-20:30 High']:
                condition = 'Above High'
            elif candle['Close'] < row['19:30-20:30 Low']:
                condition = 'Below Low'
            
            if condition:
                results.append({
                    'Date': idx.date(),
                    'First Candle Close Outside of Range': idx.time(),
                    'First Candle Close Condition': condition,
                    'First Candle Close Price': candle['Close'],
                    '19:30-20:30 High': row['19:30-20:30 High'],
                    '19:30-20:30 Low': row['19:30-20:30 Low'],
                    '20:30-16:00 High': row['20:30-16:00 High'],
                    '20:30-16:00 Low': row['20:30-16:00 Low']
                })
                break

    return pd.DataFrame(results)

# Function to track if price crosses the opposite bound after the first candle
def find_opposite_cross(df, time_ranges):
    first_candle_data = find_first_candle(df, time_ranges)
    results = []

    for _, row in first_candle_data.iterrows():
        date = row['Date']
        first_candle_time = row['First Candle Close Outside of Range']
        condition = row['First Candle Close Condition']
        high = row['19:30-20:30 High']
        low = row['19:30-20:30 Low']
        
        next_day = pd.to_datetime(date) + pd.Timedelta(days=1)
        start_range = pd.Timestamp.combine(date, first_candle_time).tz_localize('America/New_York')
        end_range = pd.Timestamp.combine(next_day, pd.Timestamp('16:00').time()).tz_localize('America/New_York')
        
        # Filter for the time range from the first candle until 4 PM the next day
        time_filtered_group = df[(df.index >= start_range) & (df.index <= end_range)]
        
        # Check for the first cross to the opposite side
        if condition == 'Above High':
            for idx, candle in time_filtered_group.iterrows():
                if candle['Close'] <= low:
                    results.append({
                        'Date': idx.date(),
                        'Break of Range Closing Time': idx.time(),
                        'Break of Low Range': 'Crossed Below Low',
                        'Break of Low Range Cross Price': candle['Close'],
                    })
                    break
        elif condition == 'Below Low':
            for idx, candle in time_filtered_group.iterrows():
                if candle['Close'] >= high:
                    results.append({
                        'Date': idx.date(),
                        'Break of Range Closing Time': idx.time(),
                        'Break of High Range': 'Crossed Above High',
                        'Break of High Range Cross Price': candle['Close'],
                    })
                    break

    return pd.DataFrame(results)


In [2]:
# All Futures assets currently downloaded for
TickerList = (
    "MES=F", "MYM=F", "NG=F", "MNQ=F", 
    "ES=F", "NQ=F", "YM=F", "CL=F", 
    "MCL=F", "GC=F", "MGC=F", "RTY=F", 
    "M2K=F", "SI=F", "PL=F", "HG=F", "SIL=F"
    )

In [3]:
for ticker in TickerList:
    df = pd.read_csv(f'Futures Asset Data/{ticker.split("=")[0]}.csv', index_col=False)
    df = df.drop(columns=['Adj Close', 'Volume'])
    df = pd.DataFrame(df)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)
    
    # Define the two time ranges
    time_ranges = [('19:30', '20:30'), ('20:30', '16:00')]

    df1 = find_high_low(df, time_ranges)
    df2 = find_first_candle(df, time_ranges)
    df3 = find_opposite_cross(df, time_ranges)

    combined_df1 = pd.merge(df1, df2, on='Date', how='left')
    combined_df1 = combined_df1.drop(columns=['19:30-20:30 High_x', '19:30-20:30 Low_x', '20:30-16:00 High_x' , '20:30-16:00 Low_x'])
    
    combined_df2 = pd.merge(combined_df1, df3, on='Date', how='left')
    combined_df2 = combined_df2.rename(columns={'19:30-20:30 High_y': '19:30-20:30 High', '19:30-20:30 Low_y': '19:30-20:30 Low',
                                                '20:30-16:00 High_y': '20:30-16:00 High', '20:30-16:00 Low_y': '20:30-16:00 Low'})


    

    # Write the .csv files
    combined_df2.to_csv(f'Backtest Results/1 Minute Timeframe Results/{ticker.split('=')[0]}.csv', index=False)

    print(ticker)

MES=F
MYM=F
NG=F
MNQ=F
ES=F
NQ=F
YM=F
CL=F
MCL=F
GC=F
MGC=F
RTY=F
M2K=F
SI=F
PL=F
HG=F
SIL=F
